# Generic Providers

## Overview

Sometimes we may want to replicate parts of a workflow and apply it to a different input.
In [Parameter Tables](parameter-tables.ipynb) we saw how we can use a parameter table for this purpose.
Another approach is to use [generics](https://mypy.readthedocs.io/en/stable/generics.html) to define a generic provider.

While parameter tables are well suited for "homogeneous" tasks such as a set of images, generics can be a better choice for "heterogeneous" tasks where the computation steps are (nearly) identical, but the purpose of the computation is different.
For example, we may have a "data" image and a "background" image (from a sensor background measurement).
We want to perform some initial homogeneous operations on both, but ultimately reach a point where we want to subtract the background from the data.

Generally speaking, using Sciline with generics can have several advantages:

- Intuitive syntax for requesting computation of intermediate results.
- Specialized providers for intermediate steps can be used instead of the necessarily identical providers when working with parameter tables.
- Maintain reusability of providers and avoid synchronization points in workflows by combining parameter tables and generic providers.

Before moving on to the next sections where we will elaborate on these points, consider how generic providers can be used straightforwardly in a pipeline.
For example, we can setup a pipeline computing a list of any type (provided that there is a parameter of the type) in a very compact manner:

In [ ]:
from typing import TypeVar, List
import sciline

T = TypeVar("T")


def duplicate(x: T) -> List[T]:
    """A generic provider that can make any list."""
    return [x, x]


pipeline = sciline.Pipeline([duplicate], params={int: 1, float: 2.0, str: "3"})

print(pipeline.compute(List[int]))
print(pipeline.compute(List[float]))
print(pipeline.compute(List[str]))

## Counter example: Naive approach

Starting with the model workflow introduced in [Getting Started](getting-started.ipynb), consider an extension where we also need to subtract a background signal from the data.
Naively, we could extend the example as follows, which is very verbose and error prone due to the duplication:

In [ ]:
from typing import NewType
import sciline

_fake_filesytem = {
    'file102.txt': [1, 2, float('nan'), 3],
    'file103.txt': [1, 2, 3, 4],
    'file104.txt': [1, 2, 3, 4, 5],
    'file105.txt': [1, 2, 3],
    'background.txt': [0.1, 0.1],
}

# 1. Define domain types

Filename = NewType('Filename', str)
BackgroundFilename = NewType('BackgroundFilename', str)
RawData = NewType('RawData', dict)
RawBackground = NewType('RawBackground', dict)
CleanedData = NewType('CleanedData', list)
CleanedBackground = NewType('CleanedBackground', list)
ScaleFactor = NewType('ScaleFactor', float)
Background = NewType('Background', list)
BackgroundSubtractedData = NewType('BackgroundSubtractedData', list)
Result = NewType('Result', float)


# 2. Define providers


def _load(filename: str) -> dict:
    data = _fake_filesytem[filename]
    return {'data': data, 'meta': {'filename': filename}}


def load(filename: Filename) -> RawData:
    """Load the data from the filename."""

    return RawData(_load(filename))


def load_background(filename: BackgroundFilename) -> RawBackground:
    """Load the background from the filename."""
    return RawBackground(_load(filename))


def _clean(raw_data: dict) -> list:
    import math

    return [x for x in raw_data['data'] if not math.isnan(x)]


def clean(raw_data: RawData) -> CleanedData:
    """Clean the data, removing NaNs."""
    return CleanedData(_clean(raw_data))


def clean_background(raw_data: RawBackground) -> CleanedBackground:
    """Clean the background, removing NaNs."""
    return CleanedBackground(_clean(raw_data))


def subtract_background(
    data: CleanedData, background: CleanedBackground
) -> BackgroundSubtractedData:
    return BackgroundSubtractedData([x - sum(background) for x in data])


def process(data: BackgroundSubtractedData, param: ScaleFactor) -> Result:
    """Process the data, multiplying the sum by the scale factor."""
    return Result(sum(data) * param)


# 3. Create pipeline

providers = [
    load,
    load_background,
    clean,
    clean_background,
    process,
    subtract_background,
]
params = {
    ScaleFactor: 2.0,
    Filename: 'file102.txt',
    BackgroundFilename: 'background.txt',
}
pipeline = sciline.Pipeline(providers, params=params)

print(f'Result={pipeline.compute(Result)}')
pipeline.visualize(Result)

We would like to reuse the `load` and `clean` functions for the background, without having to add wrappers and without duplicating all the involved domain types (`BackgroundFilename`, `RawBackgroundRaw`, and `CleanedBackground`).
However, we cannot do this directly, since `Filename`, `RawData`, and `CleanedData` are unique identifiers specific to the non-background files and the uniqueness forms the foundation of how Sciline works.

Sciline seeks to address this conundrum by providing a mechanism for using *generic providers* and for defining generic type aliases, introduced in the next section.

## Generic domain types and providers

To avoid duplicates of domain types and providers, we may instead define generic domain types and generic providers.
The example is then written as:

In [ ]:
from typing import NewType, TypeVar
import sciline

_fake_filesytem = {
    'file102.txt': [1, 2, float('nan'), 3],
    'file103.txt': [1, 2, 3, 4],
    'file104.txt': [1, 2, 3, 4, 5],
    'file105.txt': [1, 2, 3],
    'background.txt': [0.1, 0.1],
}

# 1. Define domain types

# 1.a Define generic domain types
RunType = TypeVar('RunType')


class Filename(sciline.Scope[RunType, str], str):
    ...


class RawData(sciline.Scope[RunType, dict], dict):
    ...


class CleanedData(sciline.Scope[RunType, list], list):
    ...


# 1.b Define concrete RunType values we will use.
Sample = NewType('Sample', int)
Background = NewType('Background', int)

# 1.c Define normal domain types
ScaleFactor = NewType('ScaleFactor', float)
BackgroundSubtractedData = NewType('BackgroundSubtractedData', list)
Result = NewType('Result', float)


# 2. Define providers

# 2.a Define generic providers


def load(filename: Filename[RunType]) -> RawData[RunType]:
    """Load the data from the filename."""

    data = _fake_filesytem[filename]
    return RawData[RunType]({'data': data, 'meta': {'filename': filename}})


def clean(raw_data: RawData[RunType]) -> CleanedData[RunType]:
    """Clean the data, removing NaNs."""
    import math

    return CleanedData[RunType]([x for x in raw_data['data'] if not math.isnan(x)])


# 2.b Define normal providers
def subtract_background(
    data: CleanedData[Sample], background: CleanedData[Background]
) -> BackgroundSubtractedData:
    return BackgroundSubtractedData([x - sum(background) for x in data])


def process(data: BackgroundSubtractedData, param: ScaleFactor) -> Result:
    """Process the data, multiplying the sum by the scale factor."""
    return Result(sum(data) * param)


# 3. Create pipeline

providers = [load, clean, process, subtract_background]
params = {
    ScaleFactor: 2.0,
    Filename[Sample]: 'file102.txt',
    Filename[Background]: 'background.txt',
}
pipeline = sciline.Pipeline(providers, params=params)

Apart from updated type annotations for `load` and `clean`, the code is nearly identical to the original example without background subtraction.

<div class="alert alert-info">

Note

We use a peculiar-looking syntax for defining "generic type aliases".
We would love to use [typing.NewType](https://docs.python.org/3/library/typing.html#typing.NewType) for this, but it does not allow for definition of generic aliases.
The syntax we use (subclassing [sciline.Scope](../generated/classes/sciline.Scope.rst)) is a workaround for defining generic aliases that work both at runtime and with [mypy](https://mypy-lang.org/):

```python
class Filename(sciline.Scope[RunType, str], str):
    ...
```

</div>

We can get or compute the result as usual:

In [ ]:
graph = pipeline.get(Result)
print(f'Result={graph.compute()}')
graph.visualize()

In this case, we could have achieved something similar to the above computation graph using the [Parameter Tables](parameter-tables.ipynb) feature.
In the next section we will go through the advantages of using generic providers.

## Advantages of using generic providers

### Computing intermediate results

Generic domain types with named scopes make it simple to request computation of intermediate results with a clear notation:

In [ ]:
pipeline.compute(CleanedData[Sample])

### Specialized providers

We may wish to specialize a provider for specific values of a generic's type parameters.
For example, we may need to use distinct cleaning functions for `Sample` and `Background`.
We can do so simply by defining a specialized provider for each type:

In [ ]:
def clean_data(raw_data: RawData[Sample]) -> CleanedData[Sample]:
    import math

    return CleanedData[Sample]([x for x in raw_data['data'] if not math.isnan(x)])


def clean_background(raw_data: RawData[Background]) -> CleanedData[Background]:
    return CleanedData[Background]([x for x in raw_data['data'] if not x < 0])


providers = [load, clean_data, clean_background, process, subtract_background]
pipeline = sciline.Pipeline(providers, params=params)
pipeline.visualize(Result)

### Generic providers and parameter tables

As a more complex example of where generic providers are useful, we may add a parameter table, so we can process multiple samples:

In [ ]:
RunID = NewType('RunID', int)
run_ids = [102, 103, 104, 105]
filenames = [f'file{i}.txt' for i in run_ids]
param_table = sciline.ParamTable(RunID, {Filename[Sample]: filenames}, index=run_ids)
param_table

We can now create a parametrized pipeline:

In [ ]:
params = {
    ScaleFactor: 2.0,
    Filename[Background]: 'background.txt',
}
pipeline = sciline.Pipeline(providers, params=params)
pipeline.set_param_table(param_table)
graph = pipeline.get(sciline.Series[RunID, Result])
graph.visualize()

In [ ]:
graph.compute()

With the generics mechanism, we could have added a line for the background to the parameter table.
However, the `subtrack_background` function would then have to be modified to accept a `Series[RunID, CleanedData]`.
More importantly, this would have resulted in a synchronization point in the computation graph, preventing efficient scheduling of the subsequent computation, with potentially disastrous effects on memory consumption.